<a href="https://colab.research.google.com/github/7smn2219/AI/blob/main/TF/Ranking/ranking_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-ranking
# !pip install -q --upgrade tensorflow-datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.1 which is incompatible.


In [2]:
from typing import Dict, Tuple

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr



In [3]:
# data_link = "https://grouplens.org/datasets/movielens/100k/{ratings, movies}"

ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

ratings = ratings.map(lambda x : {
    "movie_title" : x["movie_title"],
    "user_id" : x["user_id"],
    "user_rating" : x["user_rating"]
})

# movies = movies.map(lambda x : {
#     "movie_title" : x["movie_title"],
# })

In [4]:
print(ratings)
print(movies)

<_MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>
<_PrefetchDataset element_spec={'movie_genres': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [5]:
movies = movies.map(lambda x : x["movie_title"])
users = ratings.map(lambda x: x["user_id"])

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(users.batch(1000))

movies_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movies_titles_vocabulary.adapt(movies.batch(1000))

In [6]:
key_func = lambda x : user_ids_vocabulary(x["user_id"])
reduce_func = lambda key, dataset : dataset.batch(100)

ds_train = ratings.group_by_window(
    key_func=key_func,
    reduce_func=reduce_func,
    window_size=100
)

In [7]:
for x in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key} : {value.shape}")
    print(f"Example values of {key} : {value[:5].numpy()}")
    print()


'''
Shape of movie_title : (100,)
Example values of movie_title : [b'Man Who Would Be King, The (1975)' b'Silence of the Lambs, The (1991)'
 b'Next Karate Kid, The (1994)' b'2001: A Space Odyssey (1968)'
 b'Usual Suspects, The (1995)']

Shape of user_id : (100,)
Example values of user_id : [b'405' b'405' b'405' b'405' b'405']

Shape of rating : (100,)
Example values of rating : [1. 4. 1. 5. 5.]
'''

Shape of movie_title : (100,)
Example values of movie_title : [b'Man Who Would Be King, The (1975)' b'Silence of the Lambs, The (1991)'
 b'Next Karate Kid, The (1994)' b'2001: A Space Odyssey (1968)'
 b'Usual Suspects, The (1995)']

Shape of user_id : (100,)
Example values of user_id : [b'405' b'405' b'405' b'405' b'405']

Shape of user_rating : (100,)
Example values of user_rating : [1. 4. 1. 5. 5.]



"\nShape of movie_title : (100,)\nExample values of movie_title : [b'Man Who Would Be King, The (1975)' b'Silence of the Lambs, The (1991)'\n b'Next Karate Kid, The (1994)' b'2001: A Space Odyssey (1968)'\n b'Usual Suspects, The (1995)']\n\nShape of user_id : (100,)\nExample values of user_id : [b'405' b'405' b'405' b'405' b'405']\n\nShape of rating : (100,)\nExample values of rating : [1. 4. 1. 5. 5.]\n"

In [8]:
def _features_and_labels(x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
  labels = x.pop("user_rating")
  return x, labels

ds_train = ds_train.map(_features_and_labels)
ds_train = ds_train.apply(tf.data.experimental.dense_to_ragged_batch(batch_size=32))

Instructions for updating:
Use `tf.data.Dataset.ragged_batch` instead.


In [11]:
for x, labels in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key} : {value.shape}")
    print(f"Example values of {key} : {value[:3, :3].numpy()}")
    print()
  print(f"Shape of labels : {labels.shape}")
  print(f"Example values of labels : {labels[:3, :3].numpy()}")

'''
Shape of movie_title : (32, None)
Example values of movie_title : [[b'Man Who Would Be King, The (1975)'
  b'Silence of the Lambs, The (1991)' b'Next Karate Kid, The (1994)']
 [b'Flower of My Secret, The (Flor de mi secreto, La) (1995)'
  b'Little Princess, The (1939)' b'Time to Kill, A (1996)']
 [b'Kundun (1997)' b'Scream (1996)' b'Power 98 (1995)']]

Shape of user_id : (32, None)
Example values of user_id : [[b'405' b'405' b'405']
 [b'655' b'655' b'655']
 [b'13' b'13' b'13']]

Shape of labels : (32, None)
Example values of labels : [[1. 4. 1.]
 [3. 3. 3.]
 [5. 1. 1.]]
 '''

Shape of movie_title : (32, None)
Example values of movie_title : [[b'Man Who Would Be King, The (1975)'
  b'Silence of the Lambs, The (1991)' b'Next Karate Kid, The (1994)']
 [b'Flower of My Secret, The (Flor de mi secreto, La) (1995)'
  b'Little Princess, The (1939)' b'Time to Kill, A (1996)']
 [b'Kundun (1997)' b'Scream (1996)' b'Power 98 (1995)']]

Shape of user_id : (32, None)
Example values of user_id : [[b'405' b'405' b'405']
 [b'655' b'655' b'655']
 [b'13' b'13' b'13']]

Shape of labels : (32, None)
Example values of labels : [[1. 4. 1.]
 [3. 3. 3.]
 [5. 1. 1.]]


In [25]:
class MovieLensRankingModel(tf.keras.Model):

  def __init__(self, user_vocab, movie_vocab):
    super().__init__()

    # To convert string input to numeric input.
    self.user_vocab = user_vocab
    self.movie_vocab = movie_vocab
    self.user_embed = tf.keras.layers.Embedding(user_vocab.vocabulary_size(), 64)
    self.movie_embed = tf.keras.layers.Embedding(movie_vocab.vocabulary_size(), 64)

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # define how the ranking score are computed:
    # take the dot product of the user embeddings with the movie embeddings

    user_embeddings = self.user_embed(self.user_vocab(features["user_id"]))
    movie_embeddings = self.movie_embed(self.movie_vocab(features["movie_title"]))

    return tf.reduce_sum(user_embeddings * movie_embeddings, axis=2)

In [26]:
# Create the ranking model, trained with a ranking loss and evaluated with ranking matrics.
model = MovieLensRankingModel(user_ids_vocabulary, movies_titles_vocabulary)

optimizer = tf.keras.optimizers.Adagrad(0.5)
loss = tfr.keras.losses.get(
    loss = tfr.keras.losses.RankingLossKey.SOFTMAX_LOSS, ragged=True
)
eval_matrics = [
    tfr.keras.metrics.get(key="ndcg", name="metrics/ndcg", ragged=True),
    tfr.keras.metrics.get(key="mrr", name="metrics/mrr", ragged=True)
]

model.compile(optimizer=optimizer, loss=loss, metrics=eval_matrics)

In [27]:
model.fit(ds_train, epochs=3)

'''
[27]
32s
model.fit(ds_train, epochs=3)
Epoch 1/3
48/48 [==============================] - 12s 96ms/step - loss: 998.7523 - metrics/ndcg: 0.8261 - metrics/mrr: 1.0000
Epoch 2/3
48/48 [==============================] - 6s 106ms/step - loss: 997.0770 - metrics/ndcg: 0.9157 - metrics/mrr: 1.0000
Epoch 3/3
48/48 [==============================] - 6s 95ms/step - loss: 994.8379 - metrics/ndcg: 0.9375 - metrics/mrr: 1.0000
<keras.src.callbacks.History at 0x7981efb69360>
'''

Epoch 1/3
48/48 [==============================] - 12s 96ms/step - loss: 998.7523 - metrics/ndcg: 0.8261 - metrics/mrr: 1.0000
Epoch 2/3
48/48 [==============================] - 6s 106ms/step - loss: 997.0770 - metrics/ndcg: 0.9157 - metrics/mrr: 1.0000
Epoch 3/3
48/48 [==============================] - 6s 95ms/step - loss: 994.8379 - metrics/ndcg: 0.9375 - metrics/mrr: 1.0000


In [30]:
# Get moive title candidate list
for movie_titles in movies.batch(2000):
  break

# Generate the input for user 42
inputs = {
    "user_id" : tf.expand_dims(tf.repeat("42", repeats=movie_titles.shape[0]), axis=0),
    "movie_title" : tf.expand_dims(movie_titles, axis=0)
}

# Get movie recommendations for user 42
scores = model(inputs)
titles = tfr.utils.sort_by_scores(scores, [tf.expand_dims(movie_titles, axis=0)])[0]
print(f"Top 5 recommendations for user 42: {titles[0, :5]}")

'''
Top 5 recommendations for user 42:
[
  b'Toy Story (1995)'
  b'Aladdin (1992)'
  b'Star Wars (1977)'
  b"It's a Wonderful Life (1946)"
  b'Raiders of the Lost Ark (1981)'
]
 '''

Top 5 recommendations for user 42: [b'Toy Story (1995)' b'Aladdin (1992)' b'Star Wars (1977)'
 b"It's a Wonderful Life (1946)" b'Raiders of the Lost Ark (1981)']
